In [2]:
import pandas as pd
import numpy as np

In [6]:
data = pd.read_csv('../../Data//911_calls_for_service.csv',index_col='callDateTime',parse_dates=True)

In [7]:
data.drop('Unnamed: 0',axis=1,inplace=True)

In [8]:
data.head()

,priority,district,description,callNumber,incidentLocation,location
callDateTime,,,,,,
2015-07-13 10:41:00,Medium,CD,SEE TEXT,P151941002,0 N CALVERT ST,"(39.2899299,-76.6123462)"
2015-07-13 10:47:00,Medium,CD,911/NO VOICE,P151941003,600 E FAYETTE ST,"(39.2906737,-76.6071600)"
2015-07-13 10:42:00,Medium,CD,911/NO VOICE,P151941004,200 E BALTIMORE ST,"(39.2898910,-76.6120720)"
2015-07-13 10:45:00,Low,CD,PRKG COMPLAINT,P151941005,800 PARK AV,"(39.2985163,-76.6184754)"
2015-07-13 10:46:00,Medium,SW,AUTO THEFT,P151941006,3500 CLIFTON AV,"(39.3112130,-76.6763150)"


In [9]:
len(data)

2799914

In [10]:
data.priority.isnull().value_counts()

False    2793259
True        6655
Name: priority, dtype: int64

In [11]:
grouped = data.groupby(['description','priority']).count()[['callNumber']]

In [24]:
key=grouped.loc[grouped.groupby(level=0).idxmax()['callNumber'].values]

In [25]:
key.head()

,,callNumber
description,priority,
& 75,Medium,1
& 28,Medium,1
& 30,Low,1
& 49,Medium,1
& 4D,High,1


In [26]:
grouped.loc[['BURGLARY']]

callNumber
description priority                  
BURGLARY    High                    78
            Low                      2
            Medium               55470
            Out of Service           9

In [27]:
key.loc[['BURGLARY']]

,,callNumber
description,priority,
BURGLARY,Medium,55470


In [59]:
key.index.is_unique

True

In [60]:
key = key.reset_index().drop('callNumber',axis=1).set_index('description')['priority'] #key needs to be a Series

In [61]:
key.head()

description
0                 High
0171     Non-Emergency
0361     Non-Emergency
04338              Low
0600     Non-Emergency
Name: priority, dtype: object

In [62]:
key.loc['BURGLARY']

'Medium'

In [63]:
key.index.is_unique

True

In [64]:
mask = data.priority.isnull()

In [65]:
data.loc[mask,'priority'].size

6655

In [66]:
key.loc[data.loc[mask,'description']].count()

6651

In [67]:
key.loc[data.loc[mask,'description']].values

array(['Medium', 'Medium', 'High', ..., 'Medium', 'Medium', 'Low'], dtype=object)

In [68]:
data.loc[mask,'priority'] = key.loc[data.loc[mask,'description']].values

In [70]:
data.priority.isnull().value_counts()

False    2799910
True           4
Name: priority, dtype: int64

In [71]:
data[data.priority.isnull()]

,callDateTime,priority,district,description,callNumber,incidentLocation,location
1593028,2015-04-30 08:24:00,NaN,ND,CHILDREN ALONE,P151200451,3600 FALLS RD,"(39.3309611,-76.6351760)"
1802262,2015-05-05 19:42:00,NaN,CD,CK ADVISE,P151252529,300 W MADISON ST,"(39.2984330,-76.6202020)"
1818262,2015-05-11 07:09:00,NaN,NE,COURT ORDER VIOL,P151310490,BELAIR RD/LAKE AV,"(39.322656,-76.572254)"
1822727,2015-05-12 17:42:00,NaN,SW,MMEARING,P151322251,N MONROE ST/W BALTIMORE ST,"(39.288163,-76.646942)"


Unique descriptions with no priority data.<br>
Use similar descriptions to fill missing priorities.

In [72]:
data[data.description.str.contains('CHILD ALONE')]

,callDateTime,priority,district,description,callNumber,incidentLocation,location
119058,2015-08-21 10:41:00,Medium,ED,POS CHILD ALONE,P152330998,1800 N COLLINGTON AV,"(39.3110692,-76.5876606)"
157498,2015-09-04 10:26:00,Medium,ND,POSS CHILD ALONE,P152471099,2800 FOX ST,"(39.3211126,-76.6223437)"
189709,2015-12-07 08:47:00,Low,SW,CHILD ALONE,P153410596,3600 W LEXINGTON ST,"(39.2905262,-76.6772091)"
799469,2016-06-12 05:45:00,Low,SE,CHILD ALONE,P161640638,E FAYETTE ST/N PATTERSON PARK AV,"(39.294305,-76.584840)"
1772518,2015-03-21 10:17:00,Medium,SD,CHILD ALONE,P150800833,2400 SEAMON AV,"(39.2532445,-76.6242006)"
1790194,2015-05-01 07:56:00,Low,NE,CHILD ALONE,P151210433,4900 BELAIR RD,"(39.3347420,-76.5570290)"


In [73]:
data.loc[(data.description == 'CHILDREN ALONE') & (data.priority.isnull()),'priority'] = 'Medium'

In [74]:
data[data.priority.isnull()]

,callDateTime,priority,district,description,callNumber,incidentLocation,location
1802262,2015-05-05 19:42:00,NaN,CD,CK ADVISE,P151252529,300 W MADISON ST,"(39.2984330,-76.6202020)"
1818262,2015-05-11 07:09:00,NaN,NE,COURT ORDER VIOL,P151310490,BELAIR RD/LAKE AV,"(39.322656,-76.572254)"
1822727,2015-05-12 17:42:00,NaN,SW,MMEARING,P151322251,N MONROE ST/W BALTIMORE ST,"(39.288163,-76.646942)"


In [75]:
data[(data.description.str.contains('CHECK')) & (data.description.str.contains('ADVISE'))].groupby('priority').count()

,callDateTime,district,description,callNumber,incidentLocation,location
priority,,,,,,
Low,24,24,24,24,24,24
Medium,12,12,12,12,12,12


In [76]:
data.loc[(data.description == 'CK ADVISE') & (data.priority.isnull()),'priority'] = 'Low'

In [77]:
data[data.priority.isnull()]

,callDateTime,priority,district,description,callNumber,incidentLocation,location
1818262,2015-05-11 07:09:00,NaN,NE,COURT ORDER VIOL,P151310490,BELAIR RD/LAKE AV,"(39.322656,-76.572254)"
1822727,2015-05-12 17:42:00,NaN,SW,MMEARING,P151322251,N MONROE ST/W BALTIMORE ST,"(39.288163,-76.646942)"


In [78]:
data.loc[(data.description.str.contains('COURT')) & (data.description.str.contains('VIO'))].groupby('priority').count()

,callDateTime,district,description,callNumber,incidentLocation,location
priority,,,,,,
Low,19,19,19,19,19,19
Medium,1,1,1,1,1,1


In [79]:
data.loc[(data.description == 'COURT ORDER VIOL') & (data.priority.isnull()),'priority'] = 'Low'

In [80]:
data[data.priority.isnull()]

,callDateTime,priority,district,description,callNumber,incidentLocation,location
1822727,2015-05-12 17:42:00,NaN,SW,MMEARING,P151322251,N MONROE ST/W BALTIMORE ST,"(39.288163,-76.646942)"


In [81]:
data[data.description.str.contains('MMEARING')]

,callDateTime,priority,district,description,callNumber,incidentLocation,location
1822727,2015-05-12 17:42:00,NaN,SW,MMEARING,P151322251,N MONROE ST/W BALTIMORE ST,"(39.288163,-76.646942)"


In [82]:
data[data.description.str.contains('SMEARING')]

,callDateTime,priority,district,description,callNumber,incidentLocation,location


In [83]:
data.loc[(data.description == 'MMEARING') & (data.priority.isnull()),'priority'] = 'Low'

In [84]:
data.loc[data.description == 'MMEARING','description'] = 'SMEARING'

In [85]:
data.priority.isnull().value_counts()

False    2799914
Name: priority, dtype: int64

In [86]:
data.to_csv('911_calls_for_service_priority.csv')

In [87]:
data = pd.read_csv('911_calls_for_service_priority.csv')

In [88]:
data.head()

,Unnamed: 0,callDateTime,priority,district,description,callNumber,incidentLocation,location
0,0,2015-07-13 10:41:00,Medium,CD,SEE TEXT,P151941002,0 N CALVERT ST,"(39.2899299,-76.6123462)"
1,1,2015-07-13 10:47:00,Medium,CD,911 NO VOICE,P151941003,600 E FAYETTE ST,"(39.2906737,-76.6071600)"
2,2,2015-07-13 10:42:00,Medium,CD,911 NO VOICE,P151941004,200 E BALTIMORE ST,"(39.2898910,-76.6120720)"
3,3,2015-07-13 10:45:00,Low,CD,PRKG COMPLAINT,P151941005,800 PARK AV,"(39.2985163,-76.6184754)"
4,4,2015-07-13 10:46:00,Medium,SW,AUTO THEFT,P151941006,3500 CLIFTON AV,"(39.3112130,-76.6763150)"


In [89]:
data.index.is_unique

True

In [90]:
data.priority.isnull().value_counts()

False    2799914
Name: priority, dtype: int64